__No te olvides de rellenar esto:__

- Número de grupo: 2
- Nombre de los integrantes del grupo: Mario Jiménez, Alejandro Ramírez, David Seijas

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de clases mediante las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q17172850' -> {'Q101436564', 'Q1067089', 'Q186506', 'Q210970', 'Q223166', ...}, )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q5994' -> {'Q3152898', 'Q4951628', 'Q52954'})

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

Recuerda que puedes cargar cualquier estructura en formato json usando el siguiente código:

```python
import json

with open(filename, encoding='utf-8') as f:
    data = json.load(f)
```

In [1]:
import json

dic_id={}
dic_sub={}
dic_sup={}

def load(filename):
    with open(filename,encoding='utf-8') as file:
        data=json.load(file)
        for obj in data:
            linkX=obj["x"] #Obtenemos el link 
            linkY=obj["y"]
            
            idenX=linkX[31:len(linkX)] #Del link extraemos el identificador
            idenY=linkY[31:len(linkY)]
            
            dic_id.setdefault(idenX,obj["xLabel"]) #Modificamos el diccionario de identificadores
            dic_id.setdefault(idenY,obj["yLabel"])
            
            if not(idenY in dic_sub): #Modificamos el de subclases. Almacenamos las subclases en conjuntos.
                dic_sub.setdefault(idenY,{idenX})
            else:
                dic_sub[idenY].add(idenX)
                
                
            if not(idenX in dic_sup):
                dic_sup.setdefault(idenX,{idenY})
            else:
                dic_sup[idenX].add(idenY)

In [2]:
import os

#En la carpeta solo tenemos un json. Por ello, buscamos el fichero que finalice en .json
dir_actual=os.getcwd()
with os.scandir(dir_actual) as ficheros:
    for fichero in ficheros:
        if fichero.is_file() and fichero.name.endswith('.json'):
            filename=fichero.name
load(filename)

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [3]:
def print_tree(concepto,profundidad):
    print(0," ",dic_id[concepto],' (',concepto,')',sep="")
    print_tree_rec(concepto,1,profundidad)



def print_tree_rec(concepto,nivel,profundidad): #Funcion recursiva para representar el recorrido en profundidad del ejemplo
    
    #Suponemos que el concepto es el identificador y no la etiqueta
    
    if nivel<profundidad:
        if concepto in dic_sub:
            for subclase in dic_sub[concepto]:
                i=0
                while i<nivel: #Tabulador 
                    print(" ",end="")
                    i+=1
                print(nivel," ",dic_id[subclase],' (',subclase,')',sep="")
                print_tree_rec(subclase,nivel+1,profundidad)


In [4]:
def obtenerId(concepto): #Dada una etiqueta, se obtiene el identificador asociado.
    stop=False
    i=0
    iden="0"
    items=list(dic_id.items())
    while not(stop) and i<len(items):
        if items[i][1]==concepto:
            iden=items[i][0]
            stop=True
        i+=1
    return iden
        

In [5]:
concepto=obtenerId("voice") #A pesar de hacer la búsqueda con etiqueta en español, se nos guardan en ingles!
print_tree(concepto,3)

0 voice (Q17172850)
 1 male singing voice (Q54285279)
 1 alto (Q6983813)
  2 mezzosoprano ligera (Q6012300)
  2 boy alto (Q53395277)
  2 alto castrato (Q53395016)
  2 contralto (Q37137)
 1 mezzo-soprano (Q186506)
  2 altmezzo (Q682525)
  2 Q1300059 (Q1300059)
  2 light mezzo-soprano (Q54634726)
  2 sopranist (Q1999862)
  2 coloratura mezzo-soprano (Q54634572)
  2 character mezzo-soprano (Q54634862)
  2 lyric mezzo-soprano (Q1878954)
  2 dramatic mezzo-soprano (Q6012297)
  2 mezzo-soprano castrato (Q54634945)
 1 Treble voice (Q25303818)
 1 contralto (Q37137)
  2 tenorino (Q6141663)
  2 coloratura contralto (Q54635184)
  2 acting alto (Q5785182)
  2 dramatic contralto (Q5785183)
  2 deep contralto (Q54635335)
  2 lyric contralto (Q54635214)
 1 backing vocal (Q60396389)
 1 countertenor (Q223166)
  2 haute-contre (Q1873389)
 1 castrato (Q210970)
  2 alto castrato (Q53395016)
  2 soprano castrato (Q53830255)
  2 mezzo-soprano castrato (Q54634945)
 1 throat singing (Q1067089)
 1 operatic voc

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [8]:
def conceptosGenerales(concepto): 
    #Dado un concepto devuelve todas las superclases, no solo las más inmediatas (la relación de superclase es transitiva)
    
    conjunto={concepto}
    if concepto in dic_sup: #Music instrument no tiene superclase, y por lo tanto no es una clave de este diccionario.
        lista=list(dic_sup[concepto])
        i=0
        while i<len(lista):
            superclase=lista[i]
            conjunto.add(superclase)
            if superclase in dic_sup: 
                for elem in dic_sup[superclase]:
                    lista.append(elem)
            i+=1
    return conjunto




def lcs(concepto1,concepto2):
    # Calculamos los conceptos más generales de c1, que van a ser los posibles LCS. Comparamos con las superclases de c2:
    # Si encontramos una coincidencia, hacemos dos cosas: eliminamos de los posibles LCS y del conjunto solucion sus superclases.
    
    lcsResultados=set() #Pueden haber varios lcs's
    posiblesLCS=conceptosGenerales(concepto1) 
    
    if concepto2 in posiblesLCS:
        lcsResultados.add(concepto2)
    else:
        lista=list(dic_sup[concepto2])
        i=0
        while i<len(lista) and len(posiblesLCS)>0: 
            superclase=lista[i]
            if superclase in posiblesLCS: 
                lcsResultados.add(superclase)
                cg=conceptosGenerales(superclase) #conceptos más generales de un posible lcs
                #Estos conceptos nunca pueden ser lcs
                
                for elem in cg:
                    posiblesLCS.discard(elem) 
                    if elem!=superclase and elem in lcsResultados: #Elem no es LCS, es más general que superclase.
                        lcsResultados.discard(elem)
            else:
                if superclase in dic_sup:
                    for elem in dic_sup[superclase]:
                        lista.append(elem)
            i+=1
    return lcsResultados
    

In [10]:
#Pruebas:
print(lcs("Q6012297","Q54634726"))
print(lcs('Q5885030', 'Q6012300'))
print(lcs('Q27914', 'Q27914'))
print(lcs('Q15336282', 'Q64166304'))

{'Q186506'}
{'Q17172850'}
{'Q27914'}
{'Q55724840', 'Q1798603', 'Q105738'}


### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [19]:
import math

#Implementado como busqueda en profundidad con vuelta atrás.
def path(conceptoA,conceptoB):
    
    camino_minimo=[]
    minimo=math.inf
    
    
    camino_aux=[]
    aux=0
    pila=[conceptoA]
    
    while len(pila)>0:
        
        elem=pila.pop()

        #Si el elemento que se desapila no es subclase del ultimo añadido al camino auxiliar, estamos en vuelta atrás.
        while len(camino_aux)>0 and (not (elem in dic_sub[camino_aux[len(camino_aux)-1]])): #Arreglar la vuelta atrás
            aux-=1
            camino_aux.pop()
            
        aux+=1
        camino_aux.append(elem)
        
        if elem==conceptoB: #Caso final. En el else comprobamos que aux<minimo. Por lo tanto si llega aqui va a ser el minimo.
            minimo=aux
            camino_minimo=[x for x in camino_aux]
            aux-=1
            camino_aux.pop()
            
        else:
            if aux<minimo and elem in dic_sub: #Elem puede no tener subclases, de ahi la segunda condicion.
                for conc in dic_sub[elem]:
                    pila.append(conc)
            else: #Si elem no tiene subclases, vuelta atrás.
                aux-=1
                camino_aux.pop()
                
                
    return camino_minimo
    
    
    

In [33]:
#Pruebas
print(path('Q186506', 'Q54634726'))
print(path('Q17172850', 'Q6012300'))
print(path('Q27914', 'Q27914'))
print(path('Q34379', 'Q55724840'))
print(path('Q34379','Q17172850'))

['Q186506', 'Q54634726']
['Q17172850', 'Q6983813', 'Q6012300']
['Q27914']
['Q34379', 'Q1051772', 'Q19588495', 'Q1808578', 'Q30038759', 'Q55724833', 'Q55724840']
['Q34379', 'Q17172850']


### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [21]:
def similarity(conceptoA,conceptoB):
    lcsSet=lcs(conceptoA,conceptoB)
    similitud=0
    caminorootToLCS=[]
    caminoLCSToA=[]
    caminoLCSToB=[]
    for lcsElem in lcsSet:#Hallamos similitud.
        
        c1=path("Q34379",lcsElem)
        s1=len(c1)-1
        c2=path(lcsElem,conceptoA)
        s2=len(c2)-1
        c3=path(lcsElem,conceptoB)
        s3=len(c3)-1
        
        sAux=s1/(s1+s2+s3)
        if sAux>similitud:
            similitud=sAux
            caminorootToLCS=[x for x in c1]
            caminoLCSToA=[x for x in c2]
            caminoLCSToB=[x for x in c3]
    return (similitud,caminorootToLCS,caminoLCSToA,caminoLCSToB)
        

In [34]:
#Pruebas
t1=similarity('Q6012297', 'Q54634726')
print(t1[0])
print(t1[1])
print(t1[2])
print(t1[3])

0.6666666666666666
['Q34379', 'Q17172850', 'Q186506']
['Q186506']
['Q186506', 'Q54634726']


In [23]:
t2=similarity('Q186506', 'Q54634726')
print(t2[0])
print(t2[1])
print(t2[2])
print(t2[3])

0.6666666666666666
['Q34379', 'Q17172850', 'Q186506']
['Q186506']
['Q186506', 'Q54634726']


In [24]:
t3=similarity('Q27914', 'Q27914')
print(t3[0])
print(t3[1])
print(t3[2])
print(t3[3])

1.0
['Q34379', 'Q17172850', 'Q98116969', 'Q27914']
['Q27914']
['Q27914']


In [25]:
t4=similarity('Q76239', 'Q78987')
print(t4[0])
print(t4[1])
print(t4[2])
print(t4[3])

0.42857142857142855
['Q34379', 'Q1051772', 'Q1798603', 'Q230262']
['Q230262', 'Q76239']
['Q230262', 'Q57306162', 'Q6607', 'Q78987']


### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [32]:
lista=['Q5994','Q6607','Q78987','Q11405','Q8338']
resultados=[]
i=0
while i<len(lista):
    j=i+1
    while j<len(lista):
        t=similarity(lista[i],lista[j])
        print("Comparar "+ dic_id[lista[i]] + " con " + dic_id[lista[j]])
        print("Similitud: " + str(t[0]))
        print("Camino Root-LCS: " + str(t[1]))
        print("Camino LCS-C1: " + str(t[2]))
        print("Camino LCS-C2: " + str(t[3]))
        print("-------------------")
        j+=1
    i+=1
       

Comparar piano con guitar
Similitud: 0.2727272727272727
Camino Root-LCS: ['Q34379', 'Q1051772', 'Q1798603', 'Q230262']
Camino LCS-C1: ['Q230262', 'Q76239', 'Q50829016', 'Q55724736', 'Q55724742', 'Q4951628', 'Q5994']
Camino LCS-C2: ['Q230262', 'Q57306162', 'Q6607']
-------------------
Comparar piano con electric guitar
Similitud: 0.25
Camino Root-LCS: ['Q34379', 'Q1051772', 'Q1798603', 'Q230262']
Camino LCS-C1: ['Q230262', 'Q76239', 'Q50829016', 'Q55724736', 'Q55724742', 'Q4951628', 'Q5994']
Camino LCS-C2: ['Q230262', 'Q57306162', 'Q6607', 'Q78987']
-------------------
Comparar piano con flute
Similitud: 0
Camino Root-LCS: []
Camino LCS-C1: []
Camino LCS-C2: []
-------------------
Comparar piano con trumpet
Similitud: 0
Camino Root-LCS: []
Camino LCS-C1: []
Camino LCS-C2: []
-------------------
Comparar guitar con electric guitar
Similitud: 0.8333333333333334
Camino Root-LCS: ['Q34379', 'Q1051772', 'Q1798603', 'Q230262', 'Q57306162', 'Q6607']
Camino LCS-C1: ['Q6607']
Camino LCS-C2: ['Q6

### OBSERVACIONES
1. Tiene sentido que piano esté un poco más relacionado con guitarra que con guitarra eléctrica (0.27,0.25), ya que la similitud es escasa entre un piano y una guitarra; y por lo tanto más debería serlo entre un piano y un tipo de guitarra.
2. Tiene sentido que piano esté relacionado con guitarra y no con flauta pues el piano en su interior requiere de cuerdas para producir sonido. De hecho, el LCS es Q230262 - "instrumento de cuerda pulsada". Además se observa que el camino de LCS a guitarra está más proximo a piano, que tiene bastante sentido.
3. Tiene sentido que piano no esté relacionado con trompeta tampoco si no lo está con flauta.
4. Tiene sentido que guitarra o guitarra eléctrica no estén relacionados con trompeta o flauta.
5. Tiene sentido el alto valor de similitud entre guitarra y guitarra eléctrica. Quizás podríamos esperar una similitud más grande. Debido a la fórmula de similitud empleada, cuánto más especificos sean los dos términos (camino desde root a lcs mayor), mayor similitud van a tener siendo uno subclase directo del otro. Tiene sentido pensar que cuanto más específicos sean los términos, mayor similitud va a existir entre subclases. Sin embargo, pueden existir casos en los que no sea cierto: mezzosoprano y mezzosoprano ligero tienen 0.6 de similitud, esperándonos una similitud mucho mayor si comparamos con la de guitarra y guitarra eléctrica.
6. Pénsabamos que flauta y trompeta estarían un poco más relacionados, pues tan solo dista 0.13 de la similitud entre piano y guitarra.
7. En general parece una buena aproximación de la realidad.